In [ ]:
import pandas as pd
import glob
import unidecode
import re

import nltk
from nltk.corpus import stopwords

import spacy

In [ ]:
def fusion_csv(liste_csv):
    dataset = pd.read_csv(liste_csv[0])
    for ds in liste_csv[1:]:
        dataset = pd.concat([dataset, pd.read_csv(ds)], ignore_index=True)
    
    return dataset

In [ ]:
liste_csv = glob.glob('data'+os.path.sep+'*.CSV')

dataset = fusion_csv(liste_csv)

dataset

In [ ]:
def text_cleaning(text):
    text = unidecode.unidecode(text)
    text = re.sub(r"""[.,"'-:!;]""", '', text)
    text = re.sub('\s+', ' ', text)
    text = text.split(' ')
    return text

In [ ]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [ ]:
def lemmatization(nlp, texte):
    i = 0
    # On regarde chaque mot dans le texte
    # Chaque mot a le numéro i
    for mot in texte:
        # on va lemmatizer
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
    
            
    return texte

In [ ]:
def remove_https(corps):

    text_file = open(r'corps.txt', 'w',  encoding="utf-8")
    text_file.write(corps)
    text_file.close()

        
    final_text_file = open(r'final_corps.txt', 'w',  encoding="utf-8")
    reading_text_file = open(r'corps.txt', 'r',  encoding="utf-8")
    for line in reading_text_file:
        if "https" not in line:
            final_text_file.write(line)
            
    final_text_file.close()
    reading_text_file.close()

    with open('final_corps.txt', 'r', encoding="utf-8") as file:
        return file.read()

In [ ]:
def columns_treatment(df):
    df = df.iloc[:,[0,1,2,3]]
    df = df.rename(columns={'Objet':'objet','Corps':'corps','De: (nom)':'nom', 'De: (adresse)':'adresse'})
    return df

In [ ]:
def data_cleaning(df):
    
    df = columns_treatment(df)
    
    nlp_fr = spacy.load('fr_core_news_md')
    nlp_en = spacy.load('en_core_web_md')
    
    is_english = False

    for i in df.index:
        
        corps = remove_https(df['corps'][i])

        corps = text_cleaning(corps)
        
        objet = text_cleaning(df['objet'][i])

        #stop words cleaning for object
        objet = stop_words_english(objet)
        objet = stop_words_french(objet)
        
        # Lemmatization
        objet_fr = lemmatization(nlp_fr, objet)
        
        objet_en = lemmatization(nlp_en, objet)
        
        # on vérifie si l'objet a été lemmatizé en anglais ou non
        if objet_en != objet:
            is_english = True
        if is_english:
            objet = objet_en
            corps = stop_words_english(corps)
            corps = lemmatization(nlp_en, corps)
        else:
            objet = objet_fr
            corps = stop_words_french(corps)
            corps = lemmatization(nlp_fr, corps)

        df['objet'][i] = objet
        df['corps'][i] = corps

    return df

In [ ]:
dataset = data_cleaning(dataset)


In [ ]:
dataset

In [ ]:
dataset.head()